Загрузим необходимые библиотеки

In [25]:
import pandas as pd

import plotly.express as px

from sklearn.preprocessing import LabelEncoder

Загрузим данные и посмотрим на данные

In [13]:
try:
    contract = pd.read_csv('../data/contract.csv')
    internet = pd.read_csv('../data/internet.csv')
    personal = pd.read_csv('../data/personal.csv')
    phone = pd.read_csv('../data/phone.csv')
except:
    contract = pd.read_csv('/datasets/final_provider/contract.csv')
    internet = pd.read_csv('/datasets/final_provider/internet.csv')
    personal = pd.read_csv('/datasets/final_provider/personal.csv')
    phone = pd.read_csv('/datasets/final_provider/phone.csv')

# зададим имена датасетам и создадим список с табличками для удобства
contract.name = 'contract_data'
internet.name = 'internet_data'
personal.name = 'personal_data'
phone.name = 'phone_data'

data_list = [contract, internet, personal, phone]

In [14]:
for data in data_list:
    print(f"{'=' * 30} {data.name} {'='*30}")
    data.info()

============================== contract_data ==============================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB
============================== internet_data ==============================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID     


В данных отсутствуют пропуски.

In [15]:
for data in data_list:
    display(f"{'=' * 30} {data.name} {'='*30}")
    display(data.sample(10))

'============================== contract_data =============================='

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
2142,4223-BKEOR,2018-05-01,No,One year,No,Mailed check,64.85,1336.8
5484,4419-UJMUS,2014-05-01,No,Two year,Yes,Electronic check,99.35,6856.45
193,9680-NIAUV,2014-02-01,No,Two year,No,Credit card (automatic),109.70,8129.3
3473,7599-FKVXZ,2016-05-01,2019-11-01 00:00:00,Month-to-month,Yes,Bank transfer (automatic),106.15,4512.7
6673,3798-EPWRR,2019-12-01,No,Month-to-month,No,Electronic check,43.95,85.1
2961,6898-MDLZW,2019-02-01,No,Month-to-month,Yes,Mailed check,53.75,648.65
658,4464-JCOLN,2019-11-01,2020-01-01 00:00:00,One year,No,Mailed check,19.85,35.9
6134,3258-SANFR,2019-08-01,No,Month-to-month,No,Credit card (automatic),44.70,276.5
5627,9334-GWGOW,2019-07-01,No,Month-to-month,Yes,Electronic check,74.85,485.25
4763,4001-TSBTV,2015-04-01,No,Month-to-month,Yes,Electronic check,91.55,5511.65


'============================== internet_data =============================='

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
2114,8628-MFKAX,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes
1754,2482-CZGBB,Fiber optic,No,No,No,No,No,No
979,6581-NQCBA,DSL,No,No,No,Yes,Yes,Yes
1646,6784-XYJAE,DSL,No,No,Yes,No,No,No
5195,2038-YSEZE,DSL,Yes,Yes,No,Yes,No,No
852,5536-RTPWK,Fiber optic,No,Yes,No,No,Yes,Yes
1396,3055-VTCGS,DSL,No,No,Yes,No,No,Yes
1269,6032-KRXXO,Fiber optic,No,No,No,No,Yes,Yes
400,8205-OTCHB,DSL,No,Yes,Yes,No,No,Yes
1333,9365-SRSZE,Fiber optic,No,No,No,No,No,No


'============================== personal_data =============================='

,customerID,gender,SeniorCitizen,Partner,Dependents
1251,7595-EUIVN,Female,0,No,No
3968,7463-IFMQU,Female,0,Yes,No
1502,5724-BIDBU,Male,0,Yes,No
5733,4817-KEQSP,Female,0,Yes,Yes
3576,2560-QTSBS,Female,0,Yes,Yes
3429,7872-BAAZR,Female,0,Yes,Yes
5168,6000-APYLU,Male,0,No,No
2817,0936-NQLJU,Female,0,Yes,No
5861,9631-RXVJM,Male,0,No,No
1808,4816-LXZYW,Female,0,No,No


'============================== phone_data =============================='

,customerID,MultipleLines
5687,7435-ZNUYY,No
5466,4770-QAZXN,No
2114,5414-OFQCB,Yes
774,7695-PKLCZ,No
3148,9576-SYUHJ,Yes
3737,2466-FCCPT,No
3664,4256-ZWTZI,No
1459,6032-KRXXO,No
312,5160-UXJED,No
4865,6168-WFVVF,No


В данных присутствует большое количество категориальных признаков, две колонки с вещественными признаками (MonthlyCharges и TotalCharges)

In [16]:
for data in data_list:
    display(f"{'=' * 30} {data.name} {'='*30}")
    display(f'Количество строк в таблице: {data.shape[0]}')
    display(f'Количество уникальных клиентов: {len(data["customerID"].unique())}')

'============================== contract_data =============================='

'Количество строк в таблице: 7043'

'Количество уникальных клиентов: 7043'

'============================== internet_data =============================='

'Количество строк в таблице: 5517'

...

'Количество строк в таблице: 7043'

'Количество уникальных клиентов: 7043'

'============================== phone_data =============================='

'Количество строк в таблице: 6361'

'Количество уникальных клиентов: 6361'

Каждая запись в таблице содержит уникального клиента. Но количество записей в таблицах разное, значит после джоина таблиц по customerID у нас появятся пропуски в данных.

Попробуем это сделать и посмотрим на количество пропусков.
Для этого установим поле customerID в качестве индекса и сджоиним таблицы по индексу.

In [17]:
contract = contract.set_index('customerID')
personal = personal.set_index('customerID')
phone = phone.set_index('customerID')
internet = internet.set_index('customerID')

full_data = contract.join([personal, phone, internet])
full_data.name = 'full_data'

In [18]:
def get_missing_values(data: pd.DataFrame) -> None:
    """
    Выводит данные о пропусках в колонках по датафрейму.
    Не изменяет данные внутри датафрейма.

    :param data: pd.DataFrame
    :return: None
    """
    # получаем имена колонок датафрейма
    columns = data.columns.to_list()
    data_len = len(data)
    # объявляем счетчик
    counter = -1
    print('='*60)
    # если есть пропуски в данных - выводим информацию о пропусках по колонкам
    if sum(data.isnull().sum()) > 0:
        print(f'Количество записей в датафрейме {data.name}: {data_len} \n')
        print(f'В датафрейме {data.name} имеются следующие пропуски:')
        for i in data.isnull().sum():
            counter += 1
            if i > 0:
                print(f'  - в колонке {columns[counter]}: {i} пропусков, это {i/data_len:0.2%} об общего объема данных')
    else:
        print(f'Отлично, в датафрейме {data.name} отсутствуют пропуски.')

# посмотрим на пропуски в данных
get_missing_values(full_data)

Количество записей в датафрейме full_data: 7043 

В датафрейме full_data имеются следующие пропуски:
  - в колонке MultipleLines: 682 пропусков, это 9.68% об общего объема данных
  - в колонке InternetService: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке OnlineSecurity: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке OnlineBackup: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке DeviceProtection: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке TechSupport: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке StreamingTV: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке StreamingMovies: 1526 пропусков, это 21.67% об общего объема данных


In [19]:
full_data.head()

,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
customerID,,,,,,,,,,,,,,,,,,,
7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,Female,0,Yes,No,NaN,DSL,No,Yes,No,No,No,No
5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,Male,0,No,No,No,DSL,Yes,No,Yes,No,No,No
3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,No,No,No,DSL,Yes,Yes,No,No,No,No
7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,Male,0,No,No,NaN,DSL,Yes,No,Yes,Yes,No,No
9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,No,No,No,Fiber optic,No,No,No,No,No,No


В данных получилось довольно большое количество пропусков и мы не можем от них просто так избавиться.
Все пропуски присутствуют в полях с категориальными данными, так что мы можем попробовать заполнить их разными способами, а именно:
 - ввести новую категорию 'unknown' для данных с пропусками
 - заполнить пропуски самым частотным значением признака
 - посчитать доли для значений признака с пропусками, и в соответствующих значениям долях заполнить пропуски (например если у нас в признаке только значения yes или no и они распределены в долях 70% yes и 30% no, то все пропуски это 100% и из них мы рандомно заполняем 70% пропусков значением yes и 30% пропусков значением no)

Также нам необходимо создать целевой признак. Брать данные для генерации целевого признака будем из колонки EndDate таблицы contract, если no = 0, а если есть дата, то 1.
Сделаем это.


In [20]:
full_data['churn'] = full_data['EndDate'].apply(lambda x: 0 if x == 'No' else 1)

Посмотрим на распределение целевого признака.

In [21]:
full_data['churn'].value_counts(normalize=True)

0    0.73463
1    0.26537
Name: churn, dtype: float64

In [26]:
full_data.head()

,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,churn
customerID,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,Female,0,Yes,No,NaN,DSL,No,Yes,No,No,No,No,0
5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,Male,0,No,No,No,DSL,Yes,No,Yes,No,No,No,0
3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,No,No,No,DSL,Yes,Yes,No,No,No,No,1
7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,Male,0,No,No,NaN,DSL,Yes,No,Yes,Yes,No,No,0
9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,No,No,No,Fiber optic,No,No,No,No,No,No,1


Посмотрим на корреляцию с целевым признаком.

In [36]:
cat_features = ['Type', 'PaperlessBilling', 'PaymentMethod', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'MultipleLines', 'InternetService' , 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

encoder = LabelEncoder()

for feature in cat_features:
    full_data[feature] = encoder.fit_transform(full_data[feature].fillna('unknown'))

In [38]:
px.imshow(full_data.corr(), text_auto=True).show()

Видим, что целевой признак не имеет сильно скоррелированных признаков, зато признаки из таблицы internet довольно сильно скоррелированы между собой, может быть имеет смысл избавиться от этих признаков или сжать их с помощью PCA.

## Вопросы:

Что за колонки Partner и Dependents в таблице personal?

## План дальнейших действий:

 - заполнить пропуски в категориальных признаках
 - сжать данные высокоскоррелированных признаков методом РСА
 - произвести апсемплинг низкочастотного целевого признака
 - подобрать гиперпараметры и обучить разные модели классификации
 - подобрать threshold для моделей
 - сравнить метрики полученных моделей, построить conflusion matrix
 - написать выводы